<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/Bingo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SERVIDOR

In [59]:
%%writefile Servidor.py
import random
import sys
import multiprocessing.shared_memory
import time
import threading
import socket
import os

nuevoNumero = 0  # Esta variable se usa para guardar los numertos producidos
mtxProductor = threading.Semaphore(1);
mtxConsumidor = threading.Semaphore(0);
producir = True
maximos_jugadores = 5

try:
  # Para compartir los numeros salientes intento crear una MEMORIA COMPARTIDA
  mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem", create=True, size=2)
  mc.buf[0]= 0
  mc.buf[1]= 0
except FileExistsError as e:
  print("Error al crear la memoria compartida, la misma ya existe.")
  sys.exit(1)  

# Función que produce números
def productor():
  global nuevoNumero
  global producir

  # Creo una lista con todos los posibles numeros, para luego ir sacandolos y asi no se repitan
  numeros = list(range(1, 91))   
  
  while producir:
    mtxProductor.acquire()    

    if not producir:
      break

    # Tomo un numero aletatorio de la lista y lo remuevo
    nuevoNumero = random.sample(numeros, 1)      
    numeros.remove(nuevoNumero[0])      
    
    # Si la lista esta vacia
    if not numeros:      
      producir = False
              
    mtxConsumidor.release();

# Función que consume números
def consumidor():
  global nuevoNumero
  global mc  
  global cola_mensajes
  global producir
  global maximos_jugadores
  consumir = True
  bola = 1  
    
  # Para confirmar la recepcion de los datos por parte de los jugadores uso FIFO
  fifo_path = "./fifo"

  # Crear el FIFO si no existe
  if not os.path.exists(fifo_path):
    os.mkfifo(fifo_path, 0o600)

  # Abrir el FIFO en modo lectura
  fifo = os.open(fifo_path, os.O_RDONLY)
  
  while consumir:
    mtxConsumidor.acquire();       
            
    # Tomo el numero producido         
    print("Bola numero %i: %i" %(bola, nuevoNumero[0])) 
    bola += 1  
    
    # Guardo en mc el nuevo numero
    mc.buf[0] = nuevoNumero[0]   
    
    # Espero la confirmacion de recepcion de todos los jugadores
    recibidos = 0     
      
    while recibidos < maximos_jugadores:   
      mensaje = os.read(fifo, 1).decode()
      if mensaje == "":
        pass
      else:   
        recibidos += 1   

    print("Recibi confirmacion de todos los jugadores.\n")  

    # Este sleep es para que la produccion de numeros pueda verse mientras va sucediendo
    time.sleep(0.5)    

    # Si el productor termino tambien termina el consumidor
    if not producir:
      consumir = False  
    
    # verifico si ya hay un ganador, si es asi finalizo produccion y consumo
    if mc.buf[1] != 0:
      print("\nJuego finalizado, el ganador fue el jugador " + str(mc.buf[1]) + ".")  
      consumir = False
      producir = False   
         
    mtxProductor.release();  

  
  # Atrapar errores al cerrar la memoria compartida
  try:    
    mc.close()  
  except OSError as e:
    print("Error al cerrar la memoria compartida:", str(e))   
    sys.exit(1)   

  # Atrapar errores al eliminar el archivo de memoria compartida
  try:    
    mc.unlink()
  except OSError as e:
    print("Error al eliminar el archivo de memoria compartida:", str(e)) 
    sys.exit(1)  

  # Atrapar errores al cerrar el FIFO
  try:    
    os.close(fifo)  
  except OSError as e:
    print("Error al cerrar el FIFO:", str(e))
    sys.exit(1)  
    
  # Atrapar errores al eliminar el archivo FIFO
  try:
    os.unlink(fifo_path)
  except OSError as e:
    print("Error al eliminar el archivo FIFO:", str(e))  
    sys.exit(1)      

#INICIO DEL MAIN
# Esperar a que se conecten los jugadores, uso sockets para notificar su conexion
jugadores_actuales = 0

try:
  # Crea un socket
  ss = socket.socket()
  ss.bind(("127.0.0.1", 2209))    
except socket.error as e:
  print("Error al crear o conectar el socket.")
  sys.exit(1) 

# Escuchar conexiones entrantes
ss.listen(maximos_jugadores)

# Esperar a los jugadores
while jugadores_actuales < maximos_jugadores:
  # Aceptar una conexión entrante
  (cs, dir) = ss.accept()  

  # Incrementar el contador de conexiones
  jugadores_actuales += 1
  print("Se conceto el cliente numero: ", jugadores_actuales)

  # Enviar mensaje al Cliente
  mensaje = str(jugadores_actuales)
  cs.send(mensaje.encode())
  
try:
  # Cierra el socket
  cs.close()
  ss.close()
except socket.error as e:
  print("Error al cerrar el socket:", str(e))
  sys.exit(1) 

print("\nInicia el bolillero:")

# Crear los hilos
hilo_productor = threading.Thread(target=productor)
hilo_consumidor = threading.Thread(target=consumidor)

# Iniciar los hilos
hilo_productor.start()
hilo_consumidor.start()

# Esperar a que ambos hilos terminen
hilo_productor.join()
hilo_consumidor.join()
sys.exit(0)

Overwriting Servidor.py


### JUGADOR

In [71]:
%%writefile Jugador.py
import random
import sys
import multiprocessing.shared_memory
from multiprocessing.resource_tracker import unregister
import socket
import time
import os

def crear_carton():
  # Inicializar el cartón vacío
  carton = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  
  # Elegir 3 números aleatorios para cada columna del cartón
  for columna in range(9):
    if columna == 0:
      # Primer columna, números del 1 al 9
      numeros = list(range(1, 10))          
    elif columna == 8:       
      # Ultima columna, números del 80 al 90
      numeros = list(range(80, 91))
    else:
      # Resto de columnas, números del (columna * 10) al (columna * 10 + 9)
      numeros = list(range(columna * 10, columna * 10 + 10))
    
    # De los posibles numeros toma solo tres (uno por fila)
    seleccion = random.sample(numeros, 3)  

    # Los coloca en la columna de menor a mayor
    for i in range(3):
      carton[i][columna] = min(seleccion)
      seleccion.remove(min(seleccion)) 

  # Una vez completado el carton, elimina aleatoriamente 4 numeros por fila, para dejar un total de 15 numeros por carton  
  for i in range(3):    
    numeros = list(range(0, 9))           # De las nueve posibles colunas de cada fila
    descarte = random.sample(numeros, 4)  # Toma 4 aleatorias para eliminar su valor (lo pone en 0)
    for num in descarte:
      carton[i][num] = 0                 
  return carton

def imprimir_carton(carton):
  print("Cartón de Bingo:")
  for fila in carton:
    print(fila)

#INICIO DEL MAIN

time.sleep(int(sys.argv[1]))

try:
  # Intenta conectarte al servidor, usando un socket
  cs = socket.socket()
  cs.connect(("127.0.0.1", 2209))  

  # Recibir mensaje del Servidor (numero de jugador)
  mensaje = cs.recv(1024)
  jugador = mensaje.decode()
  print("Bienvenido, eres el jugador numero ", jugador)
  print("")

except ConnectionRefusedError:
  print("No se pudo establecer una conexión. El socket no está abierto o no está disponible.")
  sys.exit(1)

except socket.error as e:
  print("Ocurrió un error inesperado en el socket:", str(e))  
  sys.exit(1)

finally:
  # Cerrar el socket del cliente
  cs.close()

# Crear un cartón de bingo
carton = crear_carton()

# Imprimir el cartón
imprimir_carton(carton)

# Paso valores del carton a una lista
lista_numeros = [i for fila in carton for i in fila]

# Elimino los ceros de la lista para dejar solo los posibles valores de bolas
valor = 0
while valor in lista_numeros:
    lista_numeros.remove(valor)

try:
  # Para ver numeros que van saliendo intento acceder a la memoria compartida
  mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
  unregister(mc._name, "shared_memory")

except FileNotFoundError as e:
    print("Error al conectarse a la memoria compartida, la misma no existe.")
    sys.exit(1)

# Para notificarle al servidor la recepcion de cada valor uso FIFO
fifo_path = "./fifo"

# Crear el FIFO si no existe
if not os.path.exists(fifo_path):
  os.mkfifo(fifo_path, 0o600)

# Abrir el FIFO en modo escritura
fifo = os.open(fifo_path, os.O_WRONLY)

mensaje = str(jugador)

numero = 0  # Lo uso para comparar con el valor leido y saber si es un valor nuevo o uno ya leido anteriromente
bola = 1
consumir = True

print("\nEsperando a los demas jugadores.", end="")

while (mc.buf[0] == 0):
  pass

print("\nEstamos listo!!!\n\nSalio:")

while consumir:
  if mc.buf[1] != 0:      
    os.write(fifo, mensaje.encode()) # Notificar recepcion
    print("\nFin del juego.")  
    print("Lo sentimos, el ganador fue el jugador " + str(mc.buf[1]) + ".")      
    consumir = False
  else:
    nuevoNumero = mc.buf[0]  
    if nuevoNumero != numero:    
      numero = nuevoNumero
      print("Bola numero %i: %i" %(bola, numero))
      os.write(fifo, mensaje.encode()) # Notificar recepcion        
      bola += 1       
      if numero in lista_numeros:
        lista_numeros.remove(numero)    
      if not lista_numeros:
        print("\nFelicitaciones, ha ganado !!!")
        mc.buf[1] = int(jugador)
        consumir = False   

# Atrapar errores al cerrar la memoria compartida
try:    
  mc.close()  
except OSError as e:
  print("Error al cerrar la memoria compartida:", str(e))    

# Atrapar errores al cerrar el FIFO
try:    
  os.close(fifo)  
except OSError as e:
  print("Error al cerrar el FIFO:", str(e))

sys.exit(0)

Overwriting Jugador.py


### EJECUTAR (inicia al conectarse 5 jugadores)

Viendo la ejecucion de un Jugador:

In [80]:
!python Servidor.py 1>Servidor 2>/dev/null &

In [81]:
!python Jugador.py 0 1>Jugador1 2>/dev/null &
!python Jugador.py 0 1>Jugador2 2>/dev/null &
!python Jugador.py 0 1>Jugador3 2>/dev/null &
!python Jugador.py 0 1>Jugador4 2>/dev/null &
!python Jugador.py 0

Bienvenido, eres el jugador numero  5

Cartón de Bingo:
[0, 10, 23, 30, 0, 53, 0, 72, 0]
[4, 0, 24, 0, 44, 55, 0, 0, 84]
[0, 0, 29, 35, 45, 0, 0, 77, 87]

Esperando a los demas jugadores.
Estamos listo!!!

Salio:
Bola numero 1: 67
Bola numero 2: 79
Bola numero 3: 72
Bola numero 4: 42
Bola numero 5: 48
Bola numero 6: 58
Bola numero 7: 5
Bola numero 8: 10
Bola numero 9: 69
Bola numero 10: 24
Bola numero 11: 52
Bola numero 12: 39
Bola numero 13: 4
Bola numero 14: 89
Bola numero 15: 74
Bola numero 16: 1
Bola numero 17: 81
Bola numero 18: 53
Bola numero 19: 31
Bola numero 20: 63
Bola numero 21: 62
Bola numero 22: 83
Bola numero 23: 22
Bola numero 24: 50
Bola numero 25: 32
Bola numero 26: 44
Bola numero 27: 20
Bola numero 28: 30
Bola numero 29: 84
Bola numero 30: 73
Bola numero 31: 3
Bola numero 32: 61
Bola numero 33: 51
Bola numero 34: 21
Bola numero 35: 64
Bola numero 36: 33
Bola numero 37: 37
Bola numero 38: 87
Bola numero 39: 90
Bola numero 40: 2
Bola numero 41: 76
Bola numero 42: 35
Bol

Viendo la ejecucion del Servidor:

In [82]:
!python Jugador.py 2 1>Jugador1 2>/dev/null &
!python Jugador.py 2 1>Jugador2 2>/dev/null &
!python Jugador.py 2 1>Jugador3 2>/dev/null &
!python Jugador.py 2 1>Jugador4 2>/dev/null &
!python Jugador.py 2 1>Jugador5 2>/dev/null &
!python Servidor.py

Se conceto el cliente numero:  1
Se conceto el cliente numero:  2
Se conceto el cliente numero:  3
Se conceto el cliente numero:  4
Se conceto el cliente numero:  5

Inicia el bolillero:
Bola numero 1: 7
Recibi confirmacion de todos los jugadores.

Bola numero 2: 6
Recibi confirmacion de todos los jugadores.

Bola numero 3: 38
Recibi confirmacion de todos los jugadores.

Bola numero 4: 74
Recibi confirmacion de todos los jugadores.

Bola numero 5: 84
Recibi confirmacion de todos los jugadores.

Bola numero 6: 59
Recibi confirmacion de todos los jugadores.

Bola numero 7: 52
Recibi confirmacion de todos los jugadores.

Bola numero 8: 16
Recibi confirmacion de todos los jugadores.

Bola numero 9: 53
Recibi confirmacion de todos los jugadores.

Bola numero 10: 21
Recibi confirmacion de todos los jugadores.

Bola numero 11: 67
Recibi confirmacion de todos los jugadores.

Bola numero 12: 66
Recibi confirmacion de todos los jugadores.

Bola numero 13: 13
Recibi confirmacion de todos los juga

### EMERGENCIA CUANDO ALGO SE CERRO MAL

In [69]:
import sys
import multiprocessing.shared_memory

mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
mc.close()
mc.unlink()

In [70]:
import os
fifo_path = "./fifo"
os.unlink(fifo_path)